In [1]:
import os
import sys

import django

sys.path.append('../')  # add path to project root dir

os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
from django.apps import apps as django_apps

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

  * Reading config from flourish.ini
Loading Data Encryption (init)...
 * loading keys from /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields
 * loading rsa.restricted.public ... Done.
 * loading rsa.restricted.private ... Done.
 * loading rsa.local.public ... Done.
 * loading rsa.local.private ... Done.
 * loading aes.local ... Done.
 * loading aes.restricted ... Done.
 * loading salt.local ... Done.
 * loading salt.restricted ... Done.
 Done loading Data Encryption (init)...


/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/django_q/conf.py:139: UserWarning: Retry and timeout are misconfigured. Set retry larger than timeout, 
        failure to do so will cause the tasks to be retriggered before completion. 
        See https://django-q.readthedocs.io/en/latest/configure.html#retry for details.
  warn(
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (HistoryManagerMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (ScheduleModelMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (VisitModelMixin) without inherit=True
  warnings.wa

Loading Data Encryption ...
 * found encryption keys in /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields.
 * using model django_crypto_fields.crypt.
 Done loading Data Encryption.
Loading Edc Consent ...
 * checking for site consents ...
 * registered consents 'consents' from 'flourish_caregiver'
 * registered consents 'consents' from 'flourish_child'
 * registered consents 'consents' from 'pre_flourish'
 * registered consents 'consents' from 'flourish_facet'
 * flourish_caregiver.subjectconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 2 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 3 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbadolconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbinformedconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_child.childdummysubjectconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_child.ch

In [2]:
import json

def read_json_file(file_path):
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)
    return data

In [5]:
from docutils.parsers import null
from edc_constants.constants import OTHER
from tqdm import tqdm
from flourish_child.models import ChildHIVNotTestedReason, InfantHIVTesting

## TODO: At the time of update before applying migrations dump the data from the old table and then add the data on the variable all_infant_hiv_testing then run migrations and then run the below code to update the data
all_infant_hiv_testing = read_json_file('/Users/nimrodmunatsi/Downloads/infanthivtesting.json')
not_fould_count = 0

for infant_hiv_testing in tqdm(all_infant_hiv_testing):
    reason = infant_hiv_testing.get('fields').get('reason_child_not_tested')
    visit = infant_hiv_testing.get('fields').get('child_visit')
    reason_other = infant_hiv_testing.get('fields').get('reason_child_not_tested_other')

    try:
        live_obj=InfantHIVTesting.objects.get(
            child_visit__subject_identifier=visit[0], 
            child_visit__visit_code=visit[3],
        )
    except InfantHIVTesting.DoesNotExist:
        not_fould_count += 1
        continue
    else:
        reason_to_short_name = {
            'missed_visit': 'missed_visit',
            'hcw_decision': 'not_offered',
            'no_kits': 'no_test_kits',
            OTHER: OTHER,
            'no_apparent_reason': OTHER,
        }

        if reason in reason_to_short_name:
            obj = ChildHIVNotTestedReason.objects.get(short_name=reason_to_short_name[reason])
            live_obj.reason_child_not_tested.add(obj)
        
        if reason == OTHER or reason == 'no_apparent_reason':
            live_obj.reason_child_not_tested_other = reason_other if reason_other else 'no_apparent_reason'
        live_obj.save_base(raw=True)
        
print(not_fould_count)
        

100%|██████████| 265/265 [00:01<00:00, 264.01it/s]

39
